In [9]:
import requests
import json
import pandas as pd

In [11]:
# Requisição dos usuários na API
users = "https://fakestoreapi.com/users/"
request_users = requests.get(users)
response_users = request_users.json()

# Armazenamento em um DataFrame Pandas
df_users = pd.DataFrame(response_users)

In [12]:
# Requisição dos objetos que estão no carrinho
carts = "https://fakestoreapi.com/carts"
request_carts = requests.get(carts)
response_carts = request_carts.json()

carts_list = []
for cart in response_carts:
  for product in cart['products']:
      carts_list.append({
          "id_usuario" : cart['userId'],
          "id_produto": product['productId'],
          "quantidade": product['quantity'],
          "data": cart['date'],
      })
df_carts = pd.DataFrame(carts_list)
# Conversão data
df_carts['data'] = pd.to_datetime(df_carts['data']).dt.strftime('%Y-%m-%d')

In [13]:
# Requisição de todos os produtos
products = "https://fakestoreapi.com/products"
request_products = requests.get(products)
response_products = request_products.json()

df_products = pd.DataFrame(response_products)
df_products = df_products[['id', 'category']]
df_products = df_products.rename(columns={'category': 'categoria'})

In [14]:
# Join entre o dataframe "usuário" e "carrinho"
merged_df = pd.merge(df_users, df_carts, left_on='id', right_on='id_usuario', how='inner')

In [15]:
# Join entre o 'merged_df' com o dataframe "produtos"
merged_df_2 = pd.merge(merged_df, df_products, left_on='id_produto', right_on='id', how='inner')

In [16]:
# Somatório da quantidade de produtos e trazendo a data mais recente em cada usuário adicionou ao carrinho
grouped_df = merged_df_2.groupby(['id_usuario', 'id_produto', 'categoria']).agg(
    total_quantidade=('quantidade', 'sum'),
    ultima_data=('data', 'max')
).reset_index()

In [20]:
# Extração da data mais recente e categoria com mais produtos adicionados pelo usuário
idx = grouped_df.groupby('id_usuario')['total_quantidade'].idxmax()

final_df = grouped_df.loc[idx].reset_index(drop=True)
final_df = final_df[['id_usuario','ultima_data','categoria']]

In [22]:
# Salvar em arquivo CSV
output_path = "./fakeStorePython.csv"
final_df.to_csv(output_path, index=False)